In [1]:
import nltk
f = open('C:/Users/HP/Documents/NLP/Ezo.txt', 'r', encoding = 'utf-8')
corpus = f.read()
corpus = corpus.replace("\n\n", " ")
corpus = corpus.replace("—", " ")
corpus = corpus.replace(";", ".")
import re
corpus = re.sub('[^A-Za-z.]+', ' ', corpus)
corpus = corpus.replace(" . ", ". ")
corpus = corpus[:-1]
corpus_unpreprocessed = corpus
from nltk.tokenize import sent_tokenize
sentences = nltk.sent_tokenize(corpus)

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None
    
def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else:        
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

lemmatized_sentences = []
for sentence in sentences:
    lemmatized_sentence = lemmatize_sentence(sentence)
    lemmatized_sentences.append(lemmatized_sentence)

corpus = ' '.join(lemmatized_sentences)
corpus = corpus.lower()
words = nltk.word_tokenize(corpus)

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
final_tokens = []
for each in words:
 if each not in stop_words:
    final_tokens.append(each)
final_tokens
corpus = ' '.join(final_tokens)
corpus = corpus.replace(" . ", " ")
corpus = [corpus]

In [2]:
from nltk.corpus import gutenberg
import nltk
from operator import itemgetter

# Collocations

In [3]:
def compute_ngrams(sequence, n):
    return list(
            zip(*(sequence[index:] 
                     for index in range(n)))
    )

In [4]:
compute_ngrams([1,2,3,4], 2)

[(1, 2), (2, 3), (3, 4)]

In [5]:
compute_ngrams([1,2,3,4], 3)

[(1, 2, 3), (2, 3, 4)]

In [6]:
def flatten_corpus(corpus):
    return ' '.join([document.strip() 
                     for document in corpus])

## N-grams

In [7]:
def get_top_ngrams(corpus, ngram_val=1, limit=5):

    corpus = flatten_corpus(corpus)
    tokens = nltk.word_tokenize(corpus)

    ngrams = compute_ngrams(tokens, ngram_val)
    ngrams_freq_dist = nltk.FreqDist(ngrams)
    sorted_ngrams_fd = sorted(ngrams_freq_dist.items(), 
                              key=itemgetter(1), reverse=True)
    sorted_ngrams = sorted_ngrams_fd[0:limit]
    sorted_ngrams = [(' '.join(text), freq) 
                     for text, freq in sorted_ngrams]

    return sorted_ngrams

In [8]:
get_top_ngrams(corpus = corpus, ngram_val = 2,
               limit = 10)

[('potassium channel', 4),
 ('reward system', 3),
 ('treatment depression', 2),
 ('kcnq potassium', 2),
 ('channel within', 2),
 ('susceptible mouse', 2),
 ('within reward', 2),
 ('reward circuit', 2),
 ('depressive behavior', 2),
 ('patient mdd', 2)]

In [9]:
get_top_ngrams(corpus = corpus, ngram_val = 3,
               limit = 10)

[('kcnq potassium channel', 2),
 ('within reward circuit', 2),
 ('rest state functional', 2),
 ('depression lead cause', 1),
 ('lead cause disability', 1),
 ('cause disability worldwide', 1),
 ('disability worldwide available', 1),
 ('worldwide available treatment', 1),
 ('available treatment however', 1),
 ('treatment however partially', 1)]

## N-grams and Pointwise Mututal Information

In [10]:
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import BigramAssocMeasures

finder = BigramCollocationFinder.from_documents([item.split() 
                                                for item 
                                                in corpus])
finder

In [11]:
bigram_measures = BigramAssocMeasures()                                                
finder.nbest(bigram_measures.raw_freq, 10)

[('potassium', 'channel'),
 ('reward', 'system'),
 ('brain', 'circuitry'),
 ('channel', 'within'),
 ('depressive', 'behavior'),
 ('kcnq', 'potassium'),
 ('measure', 'use'),
 ('patient', 'mdd'),
 ('rest', 'state'),
 ('reward', 'circuit')]

In [12]:
finder.nbest(bigram_measures.pmi, 10)   

[('able', 'actively'),
 ('actively', 'counteract'),
 ('addition', 'clinical'),
 ('additional', 'limitation'),
 ('additionally', 'normalize'),
 ('administration', 'fda'),
 ('agent', 'largely'),
 ('antagonist', 'ketamine'),
 ('aspartate', 'receptor'),
 ('assess', 'specificity')]

In [13]:
from nltk.collocations import TrigramCollocationFinder
from nltk.collocations import TrigramAssocMeasures

finder = TrigramCollocationFinder.from_documents([item.split() 
                                                for item 
                                                in corpus])

In [14]:
trigram_measures = TrigramAssocMeasures()                                                
finder.nbest(trigram_measures.raw_freq, 10)

[('kcnq', 'potassium', 'channel'),
 ('rest', 'state', 'functional'),
 ('within', 'reward', 'circuit'),
 ('able', 'actively', 'counteract'),
 ('accumbens', 'nac', 'within'),
 ('action', 'base', 'decade'),
 ('action', 'undesirable', 'side'),
 ('active', 'mechanism', 'susceptible'),
 ('actively', 'counteract', 'vta'),
 ('activity', 'kcnq', 'channel')]

In [15]:
finder.nbest(trigram_measures.pmi, 10)  

[('able', 'actively', 'counteract'),
 ('addition', 'clinical', 'anhedonia'),
 ('additional', 'limitation', 'include'),
 ('administration', 'fda', 'approve'),
 ('agent', 'largely', 'share'),
 ('antagonist', 'ketamine', 'separate'),
 ('aspartate', 'receptor', 'antagonist'),
 ('cause', 'disability', 'worldwide'),
 ('cd', 'risc', 'hypothesize'),
 ('consist', 'serotonergic', 'noradrenergic')]

# Weighted Tag-based Phrase Extraction

In [16]:
sentences[:5]

['Depression is a leading cause of disability worldwide.',
 'Available treatments however are only partially effective for many patients and are associated with additional limitations including a slow onset of therapeutic action and undesirable side effects.',
 'Currently the Food and Drug Administration FDA approved treatments for depression mostly consisting of serotonergic and noradrenergic agents largely share the same basic pharmacology and mechanism of action based on decades old discoveries.',
 'This lack of mechanistic diversity leaves little opportunity for improved patient outcomes or personalized treatment approaches.',
 'In contrast rational drug discovery based on a mechanistic understanding of disease pathology promises to deliver more effective targeted therapies.']

In [17]:
import itertools
stopwords = nltk.corpus.stopwords.words('english')

def get_chunks(sentences, grammar = r'NP: {<DT>? <JJ>* <NN.*>+}', stopword_list=stopwords):
    
    all_chunks = []
    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    
    for sentence in sentences:
        
        tagged_sents = [nltk.pos_tag(nltk.word_tokenize(sentence))]   
        
        chunks = [chunker.parse(tagged_sent) 
                      for tagged_sent in tagged_sents]
        
        wtc_sents = [nltk.chunk.tree2conlltags(chunk)
                         for chunk in chunks]    
        
        flattened_chunks = list(
                            itertools.chain.from_iterable(
                                wtc_sent for wtc_sent in wtc_sents)
                           )
        
        valid_chunks_tagged = [(status, [wtc for wtc in chunk]) 
                                   for status, chunk 
                                       in itertools.groupby(flattened_chunks, 
                                                lambda word_pos_chunk: word_pos_chunk[2] != 'O')]
        
        valid_chunks = [' '.join(word.lower() 
                                for word, tag, chunk in wtc_group 
                                    if word.lower() not in stopword_list) 
                                        for status, wtc_group in valid_chunks_tagged
                                            if status]
                                            
        all_chunks.append(valid_chunks)
    
    return all_chunks

In [23]:
chunks = get_chunks(sentences)
chunks[:2]

[['depression', 'cause', 'disability worldwide'],
 ['available treatments',
  'many patients',
  'additional limitations',
  'slow onset',
  'therapeutic action',
  'undesirable side effects']]

In [19]:
from gensim import corpora, models
def get_tfidf_weighted_keyphrases(sentences, 
                                  grammar=r'NP: {<DT>? <JJ>* <NN.*>+}',
                                  top_n=10):
    valid_chunks = get_chunks(sentences, grammar=grammar)                          
    dictionary = corpora.Dictionary(valid_chunks)
    corpus = [dictionary.doc2bow(chunk) for chunk in valid_chunks]
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    weighted_phrases = {dictionary.get(idx): value 
                           for doc in corpus_tfidf 
                               for idx, value in doc}                
    weighted_phrases = sorted(weighted_phrases.items(), 
                              key=itemgetter(1), reverse=True)
    weighted_phrases = [(term, round(wt, 3)) for term, wt in weighted_phrases]
    return weighted_phrases[:top_n]

In [20]:
get_tfidf_weighted_keyphrases(sentences = sentences, top_n = 10)

[('resilience', 0.747),
 ('cause', 0.646),
 ('disability worldwide', 0.646),
 ('dopaminergic firing', 0.58),
 ('vta nac hyperactivity', 0.58),
 ('circuit', 0.577),
 ('dysregulation', 0.577),
 ('resilient animals', 0.577),
 ('contrast rational drug discovery', 0.5),
 ('disease pathology', 0.5)]

In [24]:
from gensim.summarization import keywords
key_words = keywords(corpus[0], ratio=1.0, scores=True, lemmatize=True)
[(item, round(score, 3)) for item, score in key_words][:10]

[('reward', 0.174),
 ('function strengthen resilience', 0.122),
 ('antidepressant', 0.116),
 ('patient associate', 0.103),
 ('rodent express pro depressive', 0.092),
 ('mdd connectivity', 0.083),
 ('distinct phenotype susceptible', 0.078),
 ('translational importance systemic injection ezogabine', 0.077),
 ('change', 0.076),
 ('validate mouse model', 0.076)]